In [1]:
import datetime
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.rest import REST, TimeFrame
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from trading_strategy import TradingStrategy
from data_preparation import load_data, preprocess_and_save_data, create_sequences, preprocess_data
import yaml
import pickle
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import requests
from datetime import datetime, timedelta
import time
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-06-07 22:50:22.388925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import alpaca_trade_api as tradeapi

load_dotenv()
API_KEY = os.getenv('ALPACA_API_KEY')
API_SECRET = os.getenv('ALPACA_SECRET_KEY')
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"

# api = tradeapi.REST(API_KEY, API_SECRET, APCA_API_BASE_URL, 'v2')
alpaca = REST(API_KEY, API_SECRET, base_url='https://paper-api.alpaca.markets')
api_key = os.getenv('ALPHA_VANTAGE_API_KEY')

In [2]:
with open('settings.yml', 'r') as f:
    dat = yaml.load(f, Loader=yaml.SafeLoader)

In [5]:
if len(dat['stocks'])==len(dat['stock_qty']):
    stock_qty = dict(zip(dat['stocks'],dat['stock_qty']))
else:
    print("Lengths of stocks and quantites don't match")

{'META': 5, 'TSLA': 5, 'AMZN': 5, 'MSFT': 5, 'NVDA': 5, 'AAPL': 5, 'GOOG': 6}

In [7]:
# Function to get the current position for a symbol
def get_current_position(symbol):
    try:
        position = alpaca.get_position(symbol)
        return float(position.qty)
    except:
        return 0

# Function to place a trade using Alpaca API
def place_trade(symbol, side, qty):
    alpaca.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='market',
        time_in_force='gtc'
    )

# Function to check if it's 10 minutes before market close
def is_market_closing_soon():
    clock = alpaca.get_clock()
    market_close_time = clock.next_close.replace(tzinfo=None)
    current_time = datetime.utcnow()
    return current_time >= (market_close_time - timedelta(minutes=10))

def fetch_realtime_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}&outputsize=compact"
    response = requests.get(url)
    data = response.json()
    
    if 'Time Series (1min)' in data:
        df = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index', dtype='float')
        df = df.rename(columns={"1. open": "Open", "2. high": "High", "3. low": "Low", "4. close": "Close", "5. volume": "Volume"})
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        return df
    else:
        raise ValueError("Error fetching data from Alpha Vantage")

# Function to close all positions
def close_all_positions():
    positions = alpaca.list_positions()
    for position in positions:
        side = 'sell' if float(position.qty) > 0 else 'buy'
        alpaca.submit_order(
            symbol=position.symbol,
            qty=abs(int(position.qty)),
            side=side,
            type='market',
            time_in_force='gtc'
        )

In [8]:
cur_pos = {stock:0 for stock in dat['stocks']}
model_path = '/Users/aditya/psr_v1/lstm_model_v3.keras'
model = load_model(model_path)

2024-06-07 22:43:54.363317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-07 22:43:54.365138: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-07 22:43:54.365916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
while True:
        if is_market_closing_soon():
                print("Market is closing soon, closing all positions.")
                close_all_positions()
                break
        for stock in dat['stocks']:
                data = fetch_realtime_data(stock)
                with open(f'{stock}_scaler.pkl', 'rb') as file:
                        scaler = pickle.load(file)
                processed_data = preprocess_and_save_data(data,stock)
                # Now create the sequences from the processed data
                sequences = create_sequences(processed_data.values, dat['training_duration'], dat['num_features'])
                yhat = model.predict(sequences[-1::])
                if ((scaler.inverse_transform(yhat)[0][3] - data.iloc[-1,3])/data.iloc[-1,3])>dat['profit_percent']:
                        if cur_pos[stock]<=0:
                                print("Placing a buy order.")
                                place_trade(stock, 'buy', stock_qty[stock])
                        else:
                                print("Already in a long position, no buy order placed.")
                elif ((scaler.inverse_transform(yhat)[0][3] - data.iloc[-1,3])/data.iloc[-1,3])<-dat['profit_percent']:
                        if cur_pos[stock] >= 0:  # Only sell if flat or long
                                print("Placing a sell order (shorting).")
                                place_trade(stock, 'sell', stock_qty[stock])
                        else:
                                print("Already in a short position, no sell order placed.")
        
        time.sleep(60)
        

Market is closing soon, closing all positions.
